In [1]:
from src.candidate_generator import CandidateGenerator
from src.candidate_selector import CandidateSelector
from src.llm_oracle import XiYanSQLOracle
from src.MetaSchema import MetaSchema
from MSchema.m_schema import MSchema
from MSchema.schema_engine import SchemaEngine
from sqlalchemy import create_engine
import json
import os

In [2]:
oracle = XiYanSQLOracle()
XiYan_generator = CandidateGenerator(oracle,prompt_path="template/generate_candidate_XiYan.txt")

Loading checkpoint shards:   0%|          | 0/14 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the cpu.


In [3]:
schemas = {}
for db_name in os.listdir("dev/dev_databases"):
    if(db_name.startswith(".")):
        continue
    db_path =  f'./dev/dev_databases/{db_name}/{db_name}.sqlite'
    abs_path = os.path.abspath(db_path)
    assert os.path.exists(abs_path)
    db_engine = create_engine(f'sqlite:///{abs_path}')
    schema_engine = SchemaEngine(engine=db_engine, db_name=db_name)
    mschema = schema_engine.mschema
    schemas[db_name] = mschema

In [4]:
with open("dev/dev.json") as f:
    questions = json.load(f)

In [6]:
all_json_r = []

In [ ]:
all_XiYan_r = []
for question in questions:
    schema = schemas[question["db_id"]].to_mschema()
    q = question["question"]
    h = question["evidence"]
    XiYan_r = XiYan_generator.generate_candidates(q,schema,h)
    all_XiYan_r.append(XiYan_r)
    break